# **Google Play Store Apps** - Data Cleaning

This notebook includes the procedure of cleaning data with pandas methods on google play store apps data obtained from [Kaggle](https://www.kaggle.com/datasets/lava18/google-play-store-apps).

## 1) Import Libraries

In [ ]:
# import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from IPython.display import display_html
from packaging.version import parse, Version, LegacyVersion
from packaging.utils import canonicalize_version
import os, re

## 2) Data Acquisition

Google play store apps data is acquired from [Kaggle](https://www.kaggle.com/datasets/lava18/google-play-store-apps) as two csv files *googleplaystore.csv* for app features and *googleplaystore_user_reviews.csv* for app reviews. We will load these datasets into two dataframes ***df_feature*** and ***df_review***.

#### Feature Data:

In [ ]:
# load the feature data
# df_feature = pd.read_csv('data/googleplaystore.csv', na_values = ['NaN', 'Varies with device'])
df_feature = pd.read_csv('googleplaystore.csv')

# sample feature data
df_feature.head()

In [ ]:
# observe the new dataframe details
df_feature.info(verbose=True)

In [ ]:
# missing values, show only columns with some empty
df_feature.isna().sum().iloc[df_feature.isna().sum().to_numpy().nonzero()]

In [ ]:
# missingno for features dataset, confirms the observations from previous cell
msno.bar(df_feature, figsize=(8, 4), fontsize=11)

#### Review Data:

In [ ]:
# load the review data
# df_review = pd.read_csv('data/googleplaystore_user_reviews.csv', na_values = ['nan'])
df_review=pd.read_csv('googleplaystore_user_reviews.csv')

# sample review data
df_review.head()

In [ ]:
# observe the new dataframe details
df_review.info(verbose=True)

# Observations (Gaia): the last three column are very consistent. Interesting to notice that there are 5 translated reviews
# less than the sentiment columns values, and yet still those 5 non-existent reviews have been evaluated. How is it possible?

In [ ]:
# missing values, show only columns with some empty
df_review.isna().sum().iloc[df_review.isna().sum().to_numpy().nonzero()]

In [ ]:
# missingno for features dataset, confirms the observations from previous cell
# ms.matrix(df_review)
msno.bar(df_review, figsize=(2, 4), fontsize=11)

In [ ]:
# THIS CODE CAN BE USED AFTER DATA CLEANING, DO NOT RUN FOR NOW

# save raw data for emergency uses
#if not os.path.exists('data'):
#    os.mkdir('data')
#df_feature.to_csv(os.path.join('data', 'raw_feature.csv'), encoding='utf-8-sig')
#df_review.to_csv(os.path.join('data', 'raw_review.csv'), encoding='utf-8-sig')

## 3) Data Conversion and Feature Split
We rename columns as code friendly for easier access. We will convert the 'object' type columns into str or numeric to perform imputation later by splitting the columns with multiple meaningful contents into separate columns.

In [ ]:
# rename columns to remove spaces
df_feature.columns = df_feature.columns.str.lower().str.replace(' ','_')
df_review.columns = df_review.columns.str.lower().str.replace(' ','_')

# show new column names
print("Feature:", end=" ")
print(*df_feature.columns.values, sep = ", ", end="\n")
print("Review:", end=" ")
print(*df_review.columns.values, sep = ", ", end="\n")

### 3.1) Feature Data

The columns **category**, **rating**, **type** were already available for further operations without any conversion or split. We have worked on remaining columns of Feature data as below:

#### Reviews: (Gaia)

In [ ]:
# convert to numeric by assigning errors as NaN
df_feature.reviews = df_feature.reviews.apply(pd.to_numeric, errors='coerce')

# recheck column info
df_feature.reviews.info()

#### Size: (Gaia)

In [ ]:
# remove size unit
df_feature['size'] = pd.to_numeric(df_feature['size'].replace(r"M", "", regex=True), errors='coerce')

# recheck column info
df_feature["size"].info()

In [ ]:
df_feature.iloc[[1,1450,2144,10472]]

#### Installs: (Gaia)

In [ ]:
# replace plus sign and comma with empty to remove it
df_feature.installs = pd.to_numeric(df_feature.installs.replace(('\+|,|'), '', regex=True), errors='coerce')

# recheck column info
df_feature.installs.info()

#### Price: (Beste)
All price information include a numeric value and the currency sign. Thus, we will remove the currency sign and set the price information as float.

In [ ]:
# replace currency sign with empty to remove it
df_feature.price = pd.to_numeric(df_feature.price.str.replace(r"$", "", regex=True), errors='coerce')

# recheck the previous samples
df_feature.loc[475:477]

#### Content Rating: (Beste)
Ref: https://www.esrb.org/ratings-guide/

In [ ]:
# see unique content_rating types
df_feature.content_rating.unique()

In [ ]:
# find examples where it is "Unrated"
np.where(df_feature.content_rating.str.contains("Unrated"))

In [ ]:
# replace Unrated as missing (np.nan)
df_feature.content_rating.replace('Unrated', np.NaN, inplace=True)

# recheck the previous samples
df_feature.loc[7311:7313]

In [ ]:
# see unique content_rating types
df_feature.content_rating.unique()

#### Genres: (Beste)
Since the genre includes one or more categories, we will split the column to be able to use these categorical tags. 

In [ ]:
# split genre tags as list items with delimeters ; and & and remove the items with comma (February 11, 2018)
df_feature.genres = df_feature.genres.apply(lambda x: list(filter(lambda y:re.findall(r'^[^,]+$', y), list(set(list(map(str.strip, x.replace('&',';').split(';'))))))))

# fix specific examples: "Editors", "Home" and "Educational"
df_feature.genres = df_feature.genres.apply(lambda x: list(set([y.replace('Editors', 'Video Editors').replace('Home','House').replace('Educational','Education') for y in x])))

# recheck the previous samples
df_feature.iloc[[1,1450,2144,10472]]

In [ ]:
# unique tags are below, should ve also separate at & and remove similar ones (education:educational, house:home)
df_feature.genres.explode().unique()

#### Last Updated: (Beste)

In [ ]:
# transform not unique date string of the data into datetime form
df_feature.last_updated = pd.to_datetime(df_feature.last_updated, errors='coerce')

# extract the year in a new column 'update_year'
df_feature.insert(loc=(df_feature.columns.get_loc('last_updated')),
                        column='update_year', 
                        value=df_feature.last_updated.dt.year)

# extract the month in a new column 'update_month'
df_feature.insert(loc=(df_feature.columns.get_loc('last_updated')),
                        column='update_month', 
                        value=df_feature.last_updated.dt.month)

# extract the day in a new column 'update_day'
df_feature.insert(loc=(df_feature.columns.get_loc('last_updated')),
                        column='update_day', 
                        value=df_feature.last_updated.dt.day)

# extract the day name in a new column 'update_weekday'
df_feature.insert(loc=(df_feature.columns.get_loc('last_updated')),
                        column='update_weekday', 
                        value=df_feature.last_updated.dt.day_name())

# observe generated columns with datatime content (list of col vals to get the list of related column names)
df_feature.loc[:,list(df_feature.columns.values)[10:15]].head()

#### Current Version: (Beste)

In [ ]:
# rename varying version with a shorter name
df_feature.current_ver = df_feature.current_ver.str.replace(r"Varies with device", "Varies", regex=True)

# set version string as version object (to be able to compare or sort later)
df_feature.current_ver = df_feature.current_ver.apply(lambda x: x if pd.isna(x) else parse(canonicalize_version(x)))

In [ ]:
# recheck the previous samples
df_feature.loc[7311:7314]

#### Android Version: (Beste)

In [ ]:
# rename varying version with a shorter name
df_feature.android_ver = df_feature.android_ver.str.replace(r"Varies with device", "Varies", regex=True)

# crop "and up" from version number
df_feature.android_ver = df_feature.android_ver.str.split("and up", expand=True)[0]

# set version string as version object (to be able to compare or sort later)
df_feature.android_ver = df_feature.android_ver.apply(lambda x: x if pd.isna(x) else parse(canonicalize_version(x)))

In [ ]:
# recheck the previous samples
df_feature.loc[7311:7314]

### 3.2) Review Data
All Review set columns were already available for further operations without any conversion or split. So we did not perform any column specific operation here. 

**NOTE:** *The sentiment function of textblob returns two properties, polarity, and subjectivity.*
*Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement. Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].*

## 4) Data Imputation
We check the initial empty cell rates per column within the data frame for further operations.

In [ ]:
# using missingno library to review missing values
msno.matrix(df_feature, figsize=(15, 3), fontsize=11)
msno.matrix(df_review, figsize=(15, 3), fontsize=11)

In [ ]:
# Additional plotting to see missing value percentage by columns
# TODO: parametrize the text alignment and graph colors
def findEmpty(df):
    percentages = []
    labels = []
    str_ = ""

    # iterate over data
    for col in df.columns:
        # percentage of null cells in the whole column
        p_missing = np.mean(df[col].isnull())
        # sum of null cells in the whole column
        missing = df[col].isnull().sum()
        percentages.append(p_missing*100)
        labels.append(col)
        # generate a string to print out results with numbers
        if(missing):
            str_ += "\n" + col + " - " + str(missing) + " / " + str(len(df[col]))
    return percentages, labels, str_

def plotEmpty(names, *dfs):
    # common plot calls
    text_kwargs = dict(ha='left', va='top', fontsize=11)
    plt.figure(num=None, figsize=(len(dfs)*6, 4))
    
    # color list to change color on each graph
    colors = ['skyblue','teal','pink','turquoise','maroon','navyblue']
    
    for idx, df in enumerate(dfs):
        percentages, labels, str_ = findEmpty(df)
        
        # plot results with bar plot
        ticks = list(range(1, len(labels)+1))
        plt.subplot(1, len(dfs), idx+1)
        plt.bar(ticks, percentages, color=colors[idx])
        plt.xticks(ticks, labels, rotation=75)
        plt.ylim(0, 120)
        plt.text(0.5, 125, str_, **text_kwargs)
        for index, value in enumerate(percentages):
            plt.text(index+1, value, str(int(value)), ha='center', va='bottom')
        plt.ylabel('Percentages')
        plt.title(names[idx]+' Empty Cell Percentages')
        
    plt.show()

In [ ]:
# plot empty columns of datasets
plotEmpty(["Feature", "Review"], df_feature, df_review)

Here we see that some columns of Features set have very few numbers of missing entries. For those, we will drop the rows with missing entries. For larger amount of missing entries (or not missing but unusual values as in "Varies") we will apply individual solutions.

In [ ]:
# drop rows with nan values in price, content_rating, last_updated, current_ver, android_ver 
nan_cols = df_feature.reviews.isnull() | df_feature.installs.isnull() | df_feature["type"].isnull() | df_feature.price.isnull() | df_feature.content_rating.isnull() | df_feature.last_updated.isnull() | df_feature.current_ver.isnull() | df_feature.android_ver.isnull()
df_feature.drop(df_feature[nan_cols].index, inplace=True)

# check missing values
msno.bar(df_feature, figsize=(8, 4), fontsize=11)

For reviews, if a row misses translated_review we will drop the row sice the sentiment analysis is invalid without a text.

In [ ]:
# drop rows with nan values in sentiment, sentiment_polarity and sentiment_subjectivity 
df_review.dropna(subset=['translated_review'], inplace=True)

# check missing values
msno.bar(df_review, figsize=(2, 4), fontsize=11)

After dropping the empty rows, we may now fill NaN values with column specific solutions. There is only rating column which require some imputation.

### 4.1) Feature Data

In [ ]:
# check some sample row locations for empty cells to view: np.where(df_feature.isnull())
# df_feature.iloc[np.unique(np.where(df_feature.isnull())[0])].head(3) # only rating is missing

# print some interval with empties to check the imputation results later
df_feature.loc[123:126]

#### Rating: (Gaia)

In [ ]:
# fill missing values
df_feature.rating = df_feature.rating.fillna(df_feature.rating.mean())

# recheck the previous samples
df_feature.loc[123:126]

After completing the imputation, we will check for the remaining empty cells.

In [ ]:
# check if there are any empty entries left
print("Feature:", df_feature.isnull().values.sum(), "empty entries.")
print("Review:", df_review.isnull().values.sum(), "empty entries.")

In [ ]:
# check if there are any empty entries left
plotEmpty(["Feature", "Review"], df_feature, df_review)

In [ ]:
# create a function for side by side printing of dataframes
def print_stacked(names, *dfs):
    stacked = ""
    for idx, df in enumerate(dfs):
        stacked += df.style.set_table_attributes("style='display:inline'").set_caption(names[idx])._repr_html_() + "&nbsp;"*10
    display_html(stacked, raw=True)

# selected columns for Feature set
sel_col = ["rating", "reviews", "size", "installs", "price"]

# observe numerical columns description of both tables
print_stacked(["Feature", "Review"], df_feature[sel_col].describe(), df_review.describe())

## 6) Join Data

The data sets **Feature** and **Review** include different information for common apps. Thus, we will match the titles for existing entries in both sets and obtain a final set with all the combined information.

In [ ]:
# create a copy to do not alter the original data
temp = df_feature.copy()

# drop duplicates of app in case any
temp = temp.drop_duplicates(subset=['app'])
unique_apps_f = temp['app'].tolist()

# check for duplicated entries in Feature set
print("Initially duplicated entries in Feature set:",df_feature.duplicated(subset=['app']).sum())
# check for duplicated entries
print("Remaining duplicated entries in Feature set:",temp.duplicated(subset=['app']).sum())

In [ ]:
# similarly create a copy of review set to do not alter the original data
temp_r = df_review.copy()

# drop duplicates of titles in case any
temp_r = temp_r.drop_duplicates()
unique_apps_r = temp_r['app'].tolist()

# check for duplicated entries in Review set
print("Initially duplicated entries in Review set:",df_review.duplicated().sum())
# check for duplicated entries
print("Remaining duplicated entries in Review set:",temp_r.duplicated().sum())

In [ ]:
# get a list of intersecting apps
matches = list(set(unique_apps_f) & set(unique_apps_r))

print("Unique app names with feature:",len(set(unique_apps_f)),\
      "\nUnique app names with review:", len(set(unique_apps_r)),\
      "\nMatching unique apps with features and reviews",len(matches))

In [ ]:
# filter dataframes for only matching apps
df_f_part = temp[temp.app.isin(list(matches))]
df_r_part = temp_r[temp_r.app.isin(list(matches))]

##### Merge 1: OUTER (keep repeated rows of features)

In [ ]:
# MERGE TYPE 1: OUTER MERGE (keep repeated rows of features)
result_outer = pd.merge(df_f_part, df_r_part, how="outer", on=["app", "app"])
result_outer.head()

##### Merge 2: AVERAGE (keep average of review sentiment columns)

In [ ]:
# MERGE TYPE 2: AVERAGE MERGE (keep average of review sentiment columns)
# groupby reviews on app name by taking average of numeric columns
df_r_part_avg = df_r_part.groupby("app", as_index=False).mean()

# insert sentiment by average polarity as new column
df_r_part_avg.insert(loc=(df_r_part_avg.columns.get_loc('sentiment_polarity')),\
                     column='sentiment',\
                     value=(np.where(df_r_part_avg.sentiment_polarity > 0,"Positive",\
                                     np.where(df_r_part_avg.sentiment_polarity < 0, "Negative", "Neutral"))))

# observe the averaged results for reviews
df_r_part_avg.head()

In [ ]:
# merge tables with inner to keep intersection
result_avg = pd.merge(df_f_part, df_r_part_avg, how="inner", on=["app", "app"])
result_avg.head()

In [ ]:
# store variables for dataframes in the IPython databaseto start analysis on a clear notebook
%store df_feature
%store df_review
%store result_outer
%store result_avg